# 프로젝트 Movielnes 영화 추천실습
   
     
        
        

``` wget http://files.grouplens.org/datasets/movielens/ml-1m.zip```

## libaryry import

In [1]:
import os
import pandas as pd 
import numpy as np
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

## data improt


In [2]:
#rating_file_path=os.getenv('HOME') + '/aiffel/AIFFEL_LSG/lms/E07-Recommender_sys/data/ml-1m/ratings.dat'
rating_file_path= './data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## data preprocessing

In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)


In [5]:
ratings

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [6]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'movie_id', 'count']
data = ratings[using_cols]
data.tail(10)

,user_id,movie_id,count
1000198,6040,2021,3
1000199,6040,2022,5
1000200,6040,2028,5
1000201,6040,1080,4
1000202,6040,1089,4
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4


In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
#movie_file_path=os.getenv('HOME') + '/aiffel/AIFFEL_LSG/lms/E07-Recommender_sys/data/ml-1m/movies.dat'
movie_file_path= './data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.tail(10)

,movie_id,title,genre
3873,3943,Bamboozled (2000),Comedy
3874,3944,Bootmen (2000),Comedy|Drama
3875,3945,Digimon: The Movie (2000),Adventure|Animation|Children's
3876,3946,Get Carter (2000),Action|Drama|Thriller
3877,3947,Get Carter (1971),Thriller
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


In [8]:
# 유저 이름을 보기 위해 메타 데이터를 읽어옵니다.
#movie_file_path=os.getenv('HOME') + '/aiffel/AIFFEL_LSG/lms/E07-Recommender_sys/data/ml-1m/movies.dat'
users_file_path= './data/ml-1m/users.dat'
users_cols = ['user_id', 'gender', 'age','occupation','Zip-code' ] 
users = pd.read_csv(users_file_path, sep='::', names=users_cols, engine='python')
users.tail(10)

,user_id,gender,age,occupation,Zip-code
6030,6031,F,18,0,45123
6031,6032,M,45,7,55108
6032,6033,M,50,13,78232
6033,6034,M,25,14,94117
6034,6035,F,25,1,78734
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060
6039,6040,M,25,6,11106


In [9]:
#user_unique = data['user_id'].unique()
movie_names = movies[['movie_id','title']]
movie_names

#movie_title = {val:idx for idx, val in movies['movie_id'].data,movies['title'].data}
# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
#user_to_idx = {v:k for k,v in users.items()}
#movie_to_idx = {v:k for k,v in  enumerate(artist_unique)}

,movie_id,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
...,...,...
3878,3948,Meet the Parents (2000)
3879,3949,Requiem for a Dream (2000)
3880,3950,Tigerland (2000)
3881,3951,Two Family House (2000)


In [10]:
ratings.nunique()

user_id        6039
movie_id       3628
count             3
timestamp    412911
dtype: int64

## data <분석
- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기있는 영화 30개(인기순)

In [11]:
# 영화 개수
data['movie_id'].nunique()

3628

In [12]:
# 사용자 수
data['user_id'].nunique()

6039

In [13]:
# 가장 인기 있는 영화 30개 인기수
#TODO movie id 와 movies title 매핑
movie_count= data.groupby('movie_id')['user_id'].count()
#movie_count['movie_id'].map(movies.get).dropna()
#print(movies[movies['movie_id'] == 2858]['title'])   movies[movie_count['movie_id'] == movies['movie_id']]['title']
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [14]:
print(movies[movies['movie_id'] == 2858]['title']) 

2789    American Beauty (1999)
Name: title, dtype: object


## data add 
- 내가 선호하는 영화 5가지 골라서 rating 추가


In [15]:
movies[movies["genre"]== "Action"].head(10)
#movies[movies["genre"]== "Drama"].head(30)

,movie_id,title,genre
8,9,Sudden Death (1995),Action
19,20,Money Train (1995),Action
70,71,Fair Game (1995),Action
143,145,Bad Boys (1995),Action
202,204,Under Siege 2: Dark Territory (1995),Action
224,227,Drop Zone (1994),Action
248,251,"Hunted, The (1995)",Action
312,315,"Specialist, The (1994)",Action
380,384,Bad Company (1995),Action
389,393,Street Fighter (1994),Action


In [16]:

movies[movies['title']=='Bad Company (1995)']['movie_id']


380    384
Name: movie_id, dtype: int64

In [17]:
my_favorite = ['Bad Company (1995)' , 
               'Street Fighter (1994)' ,
               'Bad Boys (1995)' ,
               'Sudden Death (1995)' ,
               'Fair Game (1995)']
my_favorite_id= [384, 393, 145, 9, 71 ]


In [18]:
data

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [19]:
ratings

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [65]:
 


my_playlist = pd.DataFrame({'user_id': [6041]*5, 
                            'movie_id':my_favorite_id, 
                            'count':4})

if not data.isin({'user_id':['6041']})['user_id'].any(): 
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 
#data = data.append(my_playlist)     
data.tail(10)

,user_id,movie_id,count
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,6041,384,4
1,6041,393,4
2,6041,145,4
3,6041,9,4
4,6041,71,4


In [70]:
data[["user_id","movie_id","count"]].to_csv("test2.csv")

In [21]:
data['user_id'].nunique()
# 1명 추가 됨...

6040

In [53]:
data.nunique()

user_id     6040
movie_id    3628
count          3
dtype: int64

In [64]:
#data['user_id'].isnull().values.sum()
#data['movie_id'].isnull().values.sum()
data.isnull().sum()

user_id     0
movie_id    0
count       0
dtype: int64

## csr matrix 

In [49]:
#from scipy.sparse import csr_matrix


num_user = data['user_id'].nunique()
num_movie = data['movie_id'].nunique()
print(num_user, num_movie)
# print(data2['count'].shape)

# print(type(data))
# print(type(num_user))
# print(type(data2.user_id))
# print(type(data2['count']))

#csr_data = csr_matrix((data['count'], (data.user_id, data.movie_id)), shape= (num_user, num_movie)) 
csr_data = csr_matrix((data['count'], (data.user_id, data.movie_id)), shape= (6050, 3955)) 
csr_data

6040 3628


<6050x3955 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [24]:
data.nunique()

user_id     6040
movie_id    3628
count          3
dtype: int64

In [25]:
# num_user = ratings['user_id'].nunique()
# num_movie = ratings['movie_id'].nunique()
# print(num_user, num_movie)
# print(ratings['count'].shape)

# csr_data = csr_matrix((ratings['count'], (ratings['user_id'], ratings['movie_id'])), shape= (num_user,num_movie))
# csr_data

## als_model = AlternatingLeastSquares 모델 훈련

In [26]:
# Implicit AlternatingLeastSquares 모델의 선언
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [27]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<4000x6050 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [28]:
# 모델 훈련
als_model.fit(csr_data_transpose)

## 선호도 파악

In [29]:
# 나의 선호도
me_idx = 6041
me_vector  = als_model.user_factors[me_idx]
print(me_vector)


[-0.1665621  -0.10275874  0.44689727 -0.1479058   0.023967    0.462207
  0.2502834  -0.02279227  0.09438989 -0.20949285 -0.09216103 -0.180838
  0.04526764  0.16610959  0.02379337 -0.00357725 -0.10907859 -0.3770728
 -0.26964825  0.05131929 -0.17688212  0.20705484 -0.21320732 -0.13183689
 -0.11719157  0.16330254  0.24423878  0.17993654 -0.01235646  0.42157263
 -0.07987457 -0.22869594 -0.31427386  0.0384491   0.06532506  0.8074599
  0.02246883 -0.13226344 -0.1430438  -0.28053337 -0.19286276  0.12383951
  0.0287216   0.2708279   0.08363315 -0.352225   -0.19985515 -0.32705307
  0.04766368 -0.3433297   0.38694856 -0.29831335 -0.09698924  0.34121293
  0.20426582 -0.03972798  0.23736015 -0.49683857 -0.43533355 -0.02248572
  0.36807573  0.12267456 -0.27025318  0.13019718  0.25813025 -0.05334624
  0.32286993  0.25396562 -0.09650721 -0.19081087  0.04461854 -0.14073004
  0.20382237 -0.16123973  0.19370931 -0.0115815  -0.27466017  0.02671979
  0.0949292  -0.13117734 -0.2249933   0.32581782 -0.16224

In [30]:
#'Fair Game (1995)'  idx 71 선호도   movie_names
fg_idx = 71
fg_vector = als_model.item_factors[fg_idx]
print(fg_vector)

np.dot(me_vector, fg_vector)

[-1.90847332e-03  2.08072423e-04  8.06523021e-03  4.78509348e-03
  3.99766769e-03  1.21818166e-02  8.68685823e-03  1.10822184e-04
  8.22159182e-03 -5.87898016e-04 -2.97538907e-04  8.52023484e-04
  4.76880372e-03  1.08619314e-02  1.02360277e-02  9.84068774e-03
  2.71388120e-03 -2.12565158e-03  5.66572882e-03  1.63922366e-03
 -6.76134741e-03  5.16194245e-03 -2.29209010e-03  8.74386053e-04
  1.05311206e-04  1.06032267e-02  2.76224758e-03  4.30257572e-03
  5.23501681e-03  6.41699741e-03 -4.76810802e-03  1.16586010e-03
 -1.04255637e-03  6.34631887e-03  4.57480922e-03  1.12881726e-02
  2.04535318e-03  8.31648591e-04  1.04885374e-03 -1.80631597e-03
  3.59573169e-04  2.44284095e-03  3.34639102e-03  8.20365082e-03
  9.75791737e-03 -3.05911247e-03  3.30506195e-03 -2.13857717e-03
  2.85179052e-03  1.40165922e-03  1.44432262e-02 -1.20749976e-03
 -4.96300578e-04  4.92422935e-03  2.99102953e-03  5.52799040e-03
  7.59191951e-03 -4.58538299e-03 -4.65775421e-03  1.42751576e-03
  9.18857288e-03  4.21785

0.07643154

In [31]:
#'Street Fighter (1994)' , idx 393 선호도   movie_names
sf_idx = 393
sf_vector = als_model.item_factors[sf_idx]
print(sf_vector)

np.dot(me_vector, sf_vector)


[-3.72860022e-03 -4.80703857e-05  4.24477691e-03  4.01358586e-03
  3.36781051e-03  1.18095279e-02  6.13664917e-04 -2.96983926e-04
  5.27595915e-03  8.59506428e-04 -9.58924531e-04 -1.04120394e-04
  5.90657024e-03  4.71514044e-03  1.00953281e-02 -1.27892505e-04
  3.48124327e-03 -2.74746912e-03  5.80488704e-03  3.37352487e-03
  6.78277633e-04  4.25438164e-03 -2.68253707e-03 -1.51120685e-03
  5.41926303e-04  3.94602027e-03  6.93815388e-03  3.51596647e-03
 -1.78567716e-03  5.94885182e-03 -2.17828376e-04 -1.04056671e-03
  2.07955833e-03  2.53827544e-03  8.25704820e-03  1.05831530e-02
  1.98491570e-03  6.47344918e-04  5.60593384e-04  2.35302933e-03
 -1.13396229e-04  2.59498344e-03 -1.71656685e-03  4.27287491e-03
  6.13539014e-03 -7.22984725e-04  2.51163146e-03 -3.52434535e-03
 -5.36633329e-03 -4.99335118e-03  5.83952945e-03 -1.40118459e-03
 -1.57928036e-03  2.04701908e-03  4.84528020e-03 -1.36762054e-03
  3.47069511e-03 -6.33018732e-04 -3.39491572e-03  2.86052586e-03
  5.19402465e-03  4.81678

0.057003513

In [32]:
#'Bad Boys' , idx 393 선호도   movie_names
bb_idx = 145
bb_vector = als_model.item_factors[bb_idx]
print(bb_vector)

np.dot(me_vector, bb_vector)
# my_favorite = ['Bad Company (1995)' , 
#                'Street Fighter (1994)' ,
#                'Bad Boys' ,
#                'Sudden Death (1995)' ,
#                'Fair Game (1995)']
# my_favorite_id= [384, 393, 145, 9, 71 ]

[ 0.01379711  0.00351854  0.02443414 -0.00032499 -0.00236257  0.01706729
  0.02340981  0.01533091  0.00523244  0.00697617  0.00306747 -0.00108967
  0.00666883  0.00755613 -0.00464258 -0.00692262 -0.00678493 -0.00594022
 -0.02067912  0.005163   -0.01193715  0.00737333  0.00617058  0.01050677
  0.0016633   0.00903852  0.01861718  0.01387041  0.00476067  0.02542521
  0.00692842  0.00065059  0.00249345 -0.00039217  0.01316982  0.03161638
  0.00128388 -0.00344346 -0.00070185 -0.00332629 -0.00990926  0.01552563
  0.00614345  0.00820123  0.00630691  0.00085048 -0.00331336 -0.0078448
  0.02284548 -0.01464618  0.0140846  -0.00743296  0.00424699  0.01747153
  0.00624067  0.01421463  0.00485581 -0.0123996  -0.01589251  0.00096436
  0.02073194  0.01199469 -0.0127747  -0.00118678  0.02120094  0.00586105
  0.01176073  0.00347592  0.02133582 -0.00399588  0.00030497 -0.01253397
  0.01384315  0.00266404  0.00673949  0.0058351  -0.01164844 -0.00537995
  0.00855932 -0.00170052 -0.00807292  0.0126118   0.

0.1843233

## 유사 영화 추천

In [33]:
#'Fair Game (1995)'  idx 71 선호도   movie_names
fg_idx = 71
similar_movies = als_model.similar_items(fg_idx, N=15)
similar_movies

[(71, 0.057454646),
 (1004, 0.050992187),
 (1744, 0.049904637),
 (544, 0.049718954),
 (1626, 0.04952762),
 (548, 0.04892122),
 (9, 0.048825465),
 (990, 0.04880291),
 (1497, 0.04878738),
 (2835, 0.04874072),
 (1869, 0.048348762),
 (20, 0.0482407),
 (704, 0.048180502),
 (886, 0.048124347),
 (694, 0.048118427)]

In [34]:
# title 매칭
for m in similar_movies:
    print(movies[movies['movie_id']==m[0]][['title','genre']])

               title   genre
70  Fair Game (1995)  Action
                       title            genre
991  Glimmer Man, The (1996)  Action|Thriller
                 title                      genre
1693  Firestorm (1998)  Action|Adventure|Thriller
                        title   genre
540  Striking Distance (1993)  Action
                       title                  genre
1583  Fire Down Below (1997)  Action|Drama|Thriller
                        title   genre
544  Terminal Velocity (1994)  Action
                 title   genre
8  Sudden Death (1995)  Action
                   title                      genre
978  Maximum Risk (1996)  Action|Adventure|Thriller
                   title   genre
1465  Double Team (1997)  Action
                    title            genre
2766  Chill Factor (1999)  Action|Thriller
                 title            genre
1800  Black Dog (1998)  Action|Thriller
                 title   genre
19  Money Train (1995)  Action
                 title            

## 좋아할 만한 영화 추천

In [35]:
#  
me_idx = 6041
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(me_idx, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(204, 0.10414383),
 (44, 0.10389449),
 (434, 0.10093868),
 (464, 0.09839),
 (1385, 0.09626536),
 (886, 0.09402799),
 (2808, 0.09375544),
 (20, 0.09091156),
 (1866, 0.09004458),
 (1100, 0.08868307),
 (1004, 0.0878624),
 (423, 0.087397784),
 (548, 0.087274134),
 (1687, 0.08647728),
 (227, 0.08629188),
 (694, 0.0833302),
 (1382, 0.08269202),
 (1639, 0.081609346),
 (3020, 0.08044555),
 (1681, 0.08034353)]

In [36]:
## 변환
a = []
# title 
for m in movie_recommended:
    a.append(movies[movies['movie_id']==m[0]]['title'])
    print(movies[movies['movie_id']==m[0]][['title','genre']])

                                    title   genre
202  Under Siege 2: Dark Territory (1995)  Action
                   title             genre
43  Mortal Kombat (1995)  Action|Adventure
                  title                   genre
430  Cliffhanger (1993)  Action|Adventure|Crime
                  title                            genre
460  Hard Target (1993)  Action|Adventure|Crime|Thriller
                   title   genre
1364  Under Siege (1992)  Action
                  title   genre
874  Bulletproof (1996)  Action
                         title          genre
2739  Universal Soldier (1992)  Action|Sci-Fi
                 title   genre
19  Money Train (1995)  Action
                    title          genre
1797  Big Hit, The (1998)  Action|Comedy
                       title           genre
1084  Days of Thunder (1990)  Action|Romance
                       title            genre
991  Glimmer Man, The (1996)  Action|Thriller
                 title            genre
419  Blown Away 

## 코멘트....
- 주입한 데이터를 액션영화 5개만을 골라서 입력해서 그런지 유사 영화와 추천영화의 장르가 전부 액션으로 나왔다.